In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").config(conf=SparkConf()).getOrCreate()

In [2]:
orders_csv = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/orders.csv')


In [3]:
customers_csv = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/customers.csv')

In [4]:
from pyspark.sql import functions as f


In [5]:
orders_text_split=orders_csv.select(
f.split(orders_csv.column_name,',')[0].cast('int').alias('order_customer_id'),
f.split(orders_csv.column_name,',')[1].cast('date').alias('order_date'),
f.split(orders_csv.column_name,',')[2].cast('int').alias('order_id'),
f.split(orders_csv.column_name,',')[3].cast('string').alias('order_status')
)

In [6]:
orders_text_split.show(5)

In [7]:
customers_text_split=customers_csv.select(
f.split(customers_csv.column_name,',')[0].cast('int').alias('customer_id'),
f.split(customers_csv.column_name,',')[1].cast('string').alias('customer_fname'),
f.split(customers_csv.column_name,',')[2].cast('string').alias('customer_lname'),
f.split(customers_csv.column_name,',')[3].cast('string').alias('customer_email'),
f.split(customers_csv.column_name,',')[4].cast('string').alias('customer_password'),
f.split(customers_csv.column_name,',')[5].cast('string').alias('customer_street'),
f.split(customers_csv.column_name,',')[6].cast('string').alias('customer_city'),
f.split(customers_csv.column_name,',')[7].cast('string').alias('customer_state'),
f.split(customers_csv.column_name,',')[8].cast('string').alias('customer_zipcode')
)

In [8]:
customers_text_split.show(5)

In [9]:
customers_text_split.join(orders_text_split,customers_text_split.customer_id==orders_text_split.order_customer_id,"inner").\
groupBy(customers_text_split.customer_state,orders_text_split.order_status).\
agg(f.count(orders_text_split.order_customer_id).alias("count")).\
filter("order_status in ('COMPLETE','CLOSED','PROCESSING')").where("count >50").\
orderBy(["customer_state","count"],ascending=[0,1]).\
select(f.format_string("%s,%s,%d","customer_state","order_status","count").alias('ANSWER')).show()
